In [1]:
# Tools
#loading Arxiv
from langchain_community.tools import ArxivQueryRun,WikipediaQueryRun
from langchain_community.utilities import ArxivAPIWrapper,WikipediaAPIWrapper

In [2]:
api_wrapper_arxiv=ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv=ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="query arxiv papers")
print(arxiv.name)

arxiv


In [3]:
arxiv.invoke("attention all you need")

'Published: 2021-05-06\nTitle: Do You Even Need Attention? A Stack of Feed-Forward Layers Does Surprisingly Well on ImageNet\nAuthors: Luke Melas-Kyriazi\nSummary: The strong performance of vision transformers on image classification and other vision tasks is often attributed to the design of their multi-head attention layers. However, the extent to which attention is responsible for this strong performance remains unclear. In this short report, we ask: is the attention layer even necessary? Specifi'

In [4]:
api_wraper_wikipedia=WikipediaAPIWrapper(top_k_results=2,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wraper_wikipedia, description="wikipedia search")
wiki.name

'wikipedia'

In [5]:
wiki.invoke("Sachin Tendulkar")

'Page: Sachin Tendulkar\nSummary: Sachin Ramesh Tendulkar ( ; Marathi: [sətɕin t̪eɳɖulkəɾ]; born 24 April 1973) is an Indian former international cricketer who captained the Indian national team. Often dubbed the "God of Cricket" in India, he is widely regarded as one of the greatest cricketers of all time. He holds several world records, including being the all-time highest run-scorer in international cricket, receiving the most player of the match awards in international cricket, and being the o'

In [6]:
from dotenv import load_dotenv
load_dotenv()

import os
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")


In [7]:
from langchain_community.tools.tavily_search import TavilySearchResults


In [8]:
tavily=TavilySearchResults()

C:\Users\parth\AppData\Local\Temp\ipykernel_23952\1844490623.py:1: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults()


In [9]:
tavily.invoke("Provide me recent AI news")

[{'title': 'AI News | Latest News | Insights Powering AI-Driven Business ...',
  'url': 'https://www.artificialintelligence-news.com/',
  'content': '### Zara’s use of AI shows how retail workflows are quietly changing\n\nDeep Dives\n\nDecember 19, 2025\n\n### Roblox brings AI into the Studio to speed up game creation\n\nAI in Action\n\nDecember 17, 2025\n\n#### Industries\n\n### Disney is embedding generative AI into its operating model\n\nEntertainment & Media\n\nDecember 24, 2025\n\n### Arm and the future of AI at the edge\n\nAI Hardware & Chips\n\nDecember 23, 2025\n\n### Marketing agencies using AI in workflows serve more clients\n\nMarketing AI [...] ### E3 2018: Xbox FastStart uses machine learning to get into content faster\n\n## Latest\n\nInside China’s push to apply AI across its energy system\n\nEnvironment & Sustainability\n\nDecember 23, 2025\n\n# Inside China’s push to apply AI across its energy system\n\nTesco signs three-year AI deal centred on customer experience\n\nRe

In [10]:
tools=[arxiv,wiki,tavily]

In [11]:
from langchain_groq import ChatGroq

In [12]:
llm=ChatGroq(model="qwen/qwen3-32b")

In [13]:
llm.invoke("what is AI")

AIMessage(content='<think>\nOkay, so I need to figure out what AI is. Let me start by recalling what I know. AI stands for Artificial Intelligence, right? But what does that actually mean? I think it\'s about machines doing things that require human intelligence. But maybe I should break it down more.\n\nFirst, what\'s the basic definition? Artificial Intelligence is a branch of computer science focused on creating systems that can perform tasks typically requiring human intelligence. That includes things like learning, reasoning, problem-solving, perception, and language understanding. But how do these systems work? Maybe there\'s a difference between narrow AI and general AI. Narrow AI is what we have now, like voice assistants or recommendation algorithms. General AI would be like a human-level intelligence across all areas, which I think is still theoretical.\n\nThen there are different techniques. Machine Learning is a subset of AI where systems learn from data. There\'s also Deep

In [14]:
llm_with_tools=llm.bind_tools(tools=tools)


In [15]:
llm_with_tools.invoke("What is recent news on AI")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for recent news on AI. Let me think about the tools I have. There\'s the Tavily search engine, which is optimized for current events. The Arxiv tool is for papers, which might be more academic and not exactly news. Wikipedia is good for general info but might not have the latest updates. So, I should use Tavily to get the most up-to-date news articles. The query should be a search term, so "recent AI news" or "latest AI developments" could work. Let me structure the tool call with that query.\n', 'tool_calls': [{'id': 'q5m5g5txm', 'function': {'arguments': '{"query":"recent AI news"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 150, 'prompt_tokens': 319, 'total_tokens': 469, 'completion_time': 0.283587407, 'completion_tokens_details': {'reasoning_tokens': 120}, 'prompt_time': 0.030157532, 'prompt_tokens_details': None, 'queue

In [16]:
llm_with_tools.invoke("What is latest research paper quantum computing")

AIMessage(content='', additional_kwargs={'reasoning_content': 'Okay, the user is asking for the latest research paper on quantum computing. I need to figure out which tool to use here. The available tools are arxiv, wikipedia, and tavily_search_results_json. \n\nArxiv is a repository for preprints in physics, math, computer science, etc., so that\'s probably the best place to look for recent research papers. Wikipedia might give general information but not the latest papers. The tavily search engine could work too, but since arxiv is specifically for academic papers, it\'s more direct.\n\nSo I should use the arxiv function with the query "quantum computing". The parameters require a "query" string. Let me make sure there are no typos. The function name is arxiv, and the argument is the search query. The user wants the latest, so maybe the most recent results from arxiv. \n\nI need to structure the tool_call correctly. The JSON should have the name as "arxiv" and the arguments as {"quer

In [17]:
#building workflow in LangGraph
from typing_extensions import TypedDict #for entire State
from langchain_core.messages import AnyMessage #human or AI message
from typing import Annotated  #labeling
from langgraph.graph.message import add_messages #add messages is reducer in langgraph

In [18]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START, END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

In [ ]:
from typing import Annotated, TypedDict
from langchain_core.messages import AnyMessage, HumanMessage
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

# 1. Define the State clearly
class GraphState(TypedDict):
    # use Annotated with add_messages to ensure tool outputs append correctly
    messages: Annotated[list[AnyMessage], add_messages]

def tool_calling_llm(state: GraphState):
    # This now correctly accesses state["messages"]
    response = llm_with_tools.invoke(state["messages"])
    return {"messages": [response]}

# 2. Build the graph
builder = StateGraph(GraphState)
builder.add_node("tool_calling_llm", tool_calling_llm)
builder.add_node("tools", ToolNode(tools))

builder.add_edge(START, "tool_calling_llm")

# 3. Use the explicit mapping in conditional edges
builder.add_conditional_edges(
    "tool_calling_llm",
    tools_condition,
    {
        "tools": "tools",
        "__end__": END
    }
)

# 4. IMPORTANT: Loop back to LLM after tool execution
builder.add_edge("tools", "tool_calling_llm")

graph_builder = builder.compile()



In [ ]:
from langchain_core.messages import HumanMessage
temp_message = graph_builder.invoke(
    {"messages": [HumanMessage(content="What is recent AI news and tell me recent research paper on Quantum computing?")]}
)

for m in temp_message["messages"]:
    print(m.pretty_print())